In [1]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.io as pio

import geopandas 
import json

In [4]:
geosonNeighbourhoods = geopandas.read_file("../../Data/neighbourhoods/neighbourhoods.geojson")
jsonGeoNeigh = json.load(open("../../Data/neighbourhoods/neighbourhoods.geojson"))

In [5]:
geosonNeighbourhoods.head()

,neighbourhood,neighbourhood_group,geometry
0,Bayswater,Queens,"MULTIPOLYGON (((-73.76671 40.61491, -73.76825 ..."
1,Allerton,Bronx,"MULTIPOLYGON (((-73.84860 40.87167, -73.84582 ..."
2,City Island,Bronx,"MULTIPOLYGON (((-73.78282 40.84392, -73.78257 ..."
3,Ditmars Steinway,Queens,"MULTIPOLYGON (((-73.90160 40.76777, -73.90391 ..."
4,Ozone Park,Queens,"MULTIPOLYGON (((-73.83754 40.69136, -73.83108 ..."


In [7]:
# from https://medium.com/@rfeers/how-to-merge-polygons-using-geopandas-for-python-merging-polygong-data-science-catalonia-7fe15eef2b14
def boundary_generator(gdf,string):
    """Devuelve el multipolígono de todos los barrios pequeños

    Args:
        gdf (GeoDataFrame): Geo DataFrame con barrios pequeños, grandes y una geometria
        string (str): Nombre del barrio grande

    Returns:
        MULTYPOLYGON: multipolygono suma
    """
    polygons = gdf.loc[(gdf['neighbourhood_group'] == string)]
    polygons = polygons['geometry'].unary_union
    return polygons


In [ ]:
for geofila